# Entrega laboratorio 3

## Integrantes 

- Javier Steven Barrera Toro - 202214779
- Julian Santiago Rolon Toloza - 202215839

In [268]:
from matplotlib import pyplot as plt
from typing import List, Tuple, NewType
import numpy as np
import pandas as pd
import sympy as sp
import tabulate

np.seterr(divide='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [19]:
print(tabulate.tabulate([['sun', '10000'], ['earth', '2000'], ['adf', 'adlkf']], headers=["Planet","R (km)", "mass (x 10^29 kg)"], tablefmt='outline'))

+----------+----------+
| Planet   | R (km)   |
+==========+==========+
| sun      | 10000    |
| earth    | 2000     |
| adf      | adlkf    |
+----------+----------+


# Problema 1: Implementación del Método Simplex Estándar

Una solución básica factible del siguiente problema de optimización puede ser $(x_1, x_2, x_3) = (0, 0, 0)$ ya que cumple con las restricciones. 

\begin{equation}
\begin{aligned}
\textrm{Maximizar} \quad & Z = 3x_1 + 2x_2 + 5x_3 \\
\textrm{s.t.} \quad & x_1 + x_2 + x_3 \leq 100 \\
  & 2x_1 + x_2 + x_3 \leq 150 \\ 
  & x_1 + 4x_2 + 2x_3 \leq 80 \\
  & x_1, x_2, x_3 \geq 0
\end{aligned}
\end{equation}

## Convertir el problema a la forma estándar

Para convertir el problema anterior en su forma estándar se deben agregar variables de holgura a las restricciones de tal manera que las mismas se vuelvan igualdades. Además, se agregan estas variables a la restricción de no negatividad y a la función objetivo.

\begin{equation}
\begin{aligned}
\textrm{Maximizar} \quad & Z = 3x_1 + 2x_2 + 5x_3 + 0s_1 + 0s_2 + 0s_3 \\
\textrm{s.t.} \quad & x_1 + x_2 + x_3 + s_1 = 100 \\
  & 2x_1 + x_2 + x_3 + s_2 = 150 \\ 
  & x_1 + 4x_2 + 2x_3 + s_3 = 80 \\
  & x_1, x_2, x_3, s_1, s_2, s_3 \geq 0
\end{aligned}
\end{equation}

## Implementación del algoritmo del método Simplex

In [285]:
class SimplexSolver:
    def __init__(self, c, X, b):
        self.z = np.array(c)
        self.X = np.array(X)
        self.b = np.array(b)

    def solve(self):
        row_num, col_num = self.X.shape
        simplex_table = np.vstack([self.z, self.X])
        c = np.hstack([[0], self.b]).reshape((row_num + 1, 1))
        
        simplex_table = np.hstack([simplex_table, c], dtype='float64')
        solutions = [simplex_table]
        
        while True:
            if np.min(simplex_table[0]) >= 0: break
            
            pivot_col = np.argmin(simplex_table[0])
            reduced_costs = simplex_table[1:, -1] / simplex_table[1:, pivot_col]

            if np.max(reduced_costs) <= 0:
                print("There's no reduced costs greater than 0")
                break

            # pivot_row = np.argmin(reduced_costs) + 1
            valid_values = reduced_costs[(reduced_costs > 0)]
            min_value = valid_values.min()
            pivot_row = np.where(reduced_costs == min_value)[0][0] + 1
            
            element = simplex_table[pivot_row, pivot_col]
            simplex_table[pivot_row] = simplex_table[pivot_row, :] / element

            for i in range(simplex_table.shape[0]):
                if i != pivot_row:
                    term = simplex_table[i, pivot_col]
                    simplex_table[i] = simplex_table[i, :] - term * simplex_table[pivot_row, :]
            
            solutions.append(simplex_table)
        
        return simplex_table[:, -1], solutions

In [284]:
c = [1, -3, -2, -5, 0, 0, 0]
b = [100, 150, 80]
X = [
    [0, 1, 1, 1, 1, 0, 0],
    [0, 2, 1, 1, 0, 1, 0],
    [0, 1, 4, 2, 0, 0, 1]
]

solver = SimplexSolver(c, X, b)
sol, _ = solver.solve()
print(sol)

[236.66666667  23.33333333  73.33333333   3.33333333]


In [283]:
c = [1, -5, -4, 0, 0, 0, 0]
X = [
    [0, 6, 4, 1, 0, 0, 0],
    [0, 1, 2, 0, 1, 0, 0],
    [0, -1, 1, 0, 0, 1, 0],
    [0, 0, 1, 0, 0, 0, 1]
]
b = [24, 6, 1, 2]

solver = SimplexSolver(c, X, b)
sol, _ = solver.solve()
print(sol)

[21.   3.   1.5  2.5  0.5]


## EXTRA: Funciones objetivo en la guia para practicar

In [282]:
X = [
    [0, 1, 2, 2, 4, 1, 0, 0],
    [0, 2, -1, 1, 2, 0, 1, 0],
    [0, 4, -2, 1, -1, 0, 0, 1]
]
b = [40, 8, 10]

zs = [
    [1, -2, -1, 3, -5, 0, 0, 0],
    [1, -8, -6, -3, 2, 0, 0, 0],
    [1, -3, 1, -3, -4, 0, 0, 0],
    [1, -5, 4, -6, 8, 0, 0, 0]
]

for z in zs:
    solver = SimplexSolver(z, X, b)
    sol, _ = solver.solve()
    print(sol)

[41.  6.  7. 29.]
[170.  15.   3.  10.]
[36.  6. 14.  8.]
[60.  6. 14.  8.]


# Problema 2: Implementación del método Simplex Dual Phase

# Problema 3: Comparación de rendimiento con GLPK/Pyomo

# Problema 4: Análisis de sensibilidad en programación lineal 